# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f16e810e550>
├── 'a' --> tensor([[-0.1509,  2.1238, -0.8176],
│                   [ 0.9425, -0.0636, -0.3239]])
└── 'x' --> <FastTreeValue 0x7f16e80f9e80>
    └── 'c' --> tensor([[-0.9007,  0.5430, -0.0387, -0.1555],
                        [-0.0841,  0.2319,  0.6464, -1.0973],
                        [ 0.1694,  0.2925,  1.3997, -0.0962]])

In [4]:
t.a

tensor([[-0.1509,  2.1238, -0.8176],
        [ 0.9425, -0.0636, -0.3239]])

In [5]:
%timeit t.a

58.9 ns ± 0.0612 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f16e810e550>
├── 'a' --> tensor([[-0.5621,  1.4266,  1.0028],
│                   [ 0.9783,  1.4707,  0.1426]])
└── 'x' --> <FastTreeValue 0x7f16e80f9e80>
    └── 'c' --> tensor([[-0.9007,  0.5430, -0.0387, -0.1555],
                        [-0.0841,  0.2319,  0.6464, -1.0973],
                        [ 0.1694,  0.2925,  1.3997, -0.0962]])

In [7]:
%timeit t.a = new_value

61.2 ns ± 0.0518 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1509,  2.1238, -0.8176],
               [ 0.9425, -0.0636, -0.3239]]),
    x: Batch(
           c: tensor([[-0.9007,  0.5430, -0.0387, -0.1555],
                      [-0.0841,  0.2319,  0.6464, -1.0973],
                      [ 0.1694,  0.2925,  1.3997, -0.0962]]),
       ),
)

In [10]:
b.a

tensor([[-0.1509,  2.1238, -0.8176],
        [ 0.9425, -0.0636, -0.3239]])

In [11]:
%timeit b.a

52.7 ns ± 0.0561 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.1671,  0.1571,  1.1405],
               [-0.0725,  0.8347,  1.2019]]),
    x: Batch(
           c: tensor([[-0.9007,  0.5430, -0.0387, -0.1555],
                      [-0.0841,  0.2319,  0.6464, -1.0973],
                      [ 0.1694,  0.2925,  1.3997, -0.0962]]),
       ),
)

In [13]:
%timeit b.a = new_value

478 ns ± 0.158 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

832 ns ± 9.19 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 39 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 544 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 398 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f16313d5be0>
├── 'a' --> tensor([[[-0.1509,  2.1238, -0.8176],
│                    [ 0.9425, -0.0636, -0.3239]],
│           
│                   [[-0.1509,  2.1238, -0.8176],
│                    [ 0.9425, -0.0636, -0.3239]],
│           
│                   [[-0.1509,  2.1238, -0.8176],
│                    [ 0.9425, -0.0636, -0.3239]],
│           
│                   [[-0.1509,  2.1238, -0.8176],
│                    [ 0.9425, -0.0636, -0.3239]],
│           
│                   [[-0.1509,  2.1238, -0.8176],
│                    [ 0.9425, -0.0636, -0.3239]],
│           
│                   [[-0.1509,  2.1238, -0.8176],
│                    [ 0.9425, -0.0636, -0.3239]],
│           
│                   [[-0.1509,  2.1238, -0.8176],
│                    [ 0.9425, -0.0636, -0.3239]],
│           
│                   [[-0.1509,  2.1238, -0.8176],
│                    [ 0.9425, -0.0636, -0.3239]]])
└── 'x' --> <FastTreeValue 0x7f16313d5af0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 66.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f16e80a1d60>
├── 'a' --> tensor([[-0.1509,  2.1238, -0.8176],
│                   [ 0.9425, -0.0636, -0.3239],
│                   [-0.1509,  2.1238, -0.8176],
│                   [ 0.9425, -0.0636, -0.3239],
│                   [-0.1509,  2.1238, -0.8176],
│                   [ 0.9425, -0.0636, -0.3239],
│                   [-0.1509,  2.1238, -0.8176],
│                   [ 0.9425, -0.0636, -0.3239],
│                   [-0.1509,  2.1238, -0.8176],
│                   [ 0.9425, -0.0636, -0.3239],
│                   [-0.1509,  2.1238, -0.8176],
│                   [ 0.9425, -0.0636, -0.3239],
│                   [-0.1509,  2.1238, -0.8176],
│                   [ 0.9425, -0.0636, -0.3239],
│                   [-0.1509,  2.1238, -0.8176],
│                   [ 0.9425, -0.0636, -0.3239]])
└── 'x' --> <FastTreeValue 0x7f16313d5c70>
    └── 'c' --> tensor([[-0.9007,  0.5430, -0.0387, -0.1555],
                        [-0.0841,  0.2319,  0.6464, -1.0973],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 78.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.2 µs ± 53.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.1509,  2.1238, -0.8176],
                [ 0.9425, -0.0636, -0.3239]],
       
               [[-0.1509,  2.1238, -0.8176],
                [ 0.9425, -0.0636, -0.3239]],
       
               [[-0.1509,  2.1238, -0.8176],
                [ 0.9425, -0.0636, -0.3239]],
       
               [[-0.1509,  2.1238, -0.8176],
                [ 0.9425, -0.0636, -0.3239]],
       
               [[-0.1509,  2.1238, -0.8176],
                [ 0.9425, -0.0636, -0.3239]],
       
               [[-0.1509,  2.1238, -0.8176],
                [ 0.9425, -0.0636, -0.3239]],
       
               [[-0.1509,  2.1238, -0.8176],
                [ 0.9425, -0.0636, -0.3239]],
       
               [[-0.1509,  2.1238, -0.8176],
                [ 0.9425, -0.0636, -0.3239]]]),
    x: Batch(
           c: tensor([[[-0.9007,  0.5430, -0.0387, -0.1555],
                       [-0.0841,  0.2319,  0.6464, -1.0973],
                       [ 0.1694,  0.2925,  1.3997, -0.0962]],
         

In [26]:
%timeit Batch.stack(batches)

77.6 µs ± 177 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.1509,  2.1238, -0.8176],
               [ 0.9425, -0.0636, -0.3239],
               [-0.1509,  2.1238, -0.8176],
               [ 0.9425, -0.0636, -0.3239],
               [-0.1509,  2.1238, -0.8176],
               [ 0.9425, -0.0636, -0.3239],
               [-0.1509,  2.1238, -0.8176],
               [ 0.9425, -0.0636, -0.3239],
               [-0.1509,  2.1238, -0.8176],
               [ 0.9425, -0.0636, -0.3239],
               [-0.1509,  2.1238, -0.8176],
               [ 0.9425, -0.0636, -0.3239],
               [-0.1509,  2.1238, -0.8176],
               [ 0.9425, -0.0636, -0.3239],
               [-0.1509,  2.1238, -0.8176],
               [ 0.9425, -0.0636, -0.3239]]),
    x: Batch(
           c: tensor([[-0.9007,  0.5430, -0.0387, -0.1555],
                      [-0.0841,  0.2319,  0.6464, -1.0973],
                      [ 0.1694,  0.2925,  1.3997, -0.0962],
                      [-0.9007,  0.5430, -0.0387, -0.1555],
                      [-0.0841,  

In [28]:
%timeit Batch.cat(batches)

142 µs ± 222 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

309 µs ± 610 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
